In [98]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: '%.6f' % x)

MAX_YEAR = 3.0
DEL_UNDER_YEAR = 2 * 4  # 12 quaters

ffill_lag = lambda col: col.where(col.notna(), col.shift(1))
bfill_lag = lambda col: col.where(col.notna(), col.shift(-1))

##### Manipulation
- ffill bfill on [asset, sales, op_earn, depre_amort]
- main_quater
  - MAX_YEAR=5 -> 3 years
  - pass when missing "only once"
- bye bye continuity...
  - supplier financing avg...
  - delta cashflow avg...

In [2]:
# dfs = {
#     "ind_code": pd.read_excel("./dataguide/총자산.xlsx", sheet_name="ind_code"),
#     "op_earn": pd.read_excel("./dataguide/김도연1.xlsx", sheet_name="op_earn"),
#     "depre_amort": pd.read_excel("./dataguide/김도연1.xlsx", sheet_name="depre_amort"),
#     "asset": pd.read_excel("./dataguide/총자산.xlsx", sheet_name="asset"),
#     "exchange": pd.read_excel("./dataguide/총자산.xlsx", sheet_name="exchange")
# }

In [3]:
# 제공된 파일 및 시트 구조
file_sheets = {
    "김도연1.xlsx": ['depre_amort', 'depre', 'age', 'n_acq_intangi'],
    "김도연2.xlsx": ['tangi', 'intangi', 'cash'],
    "한민지.xlsx": ['payab', 'invent', 'const_receiv2'],
    "오영식.xlsx": ['receiv', 'st_debt', 'lt_debt', 't_debt', 'const_receiv1'],
    "dataguide_1211.xlsx": ['amort', 'defer_tax_debt', 'common_mktcap', 'prefer_mktcap',
        'asset', 'exchange', 'ind_code', 'sales', 'op_earn', 'n_acq_tangi']
}

dfs = {}

for file, sheets in file_sheets.items():
    for sheet in sheets:
        try:
            if sheet not in dfs:
                dfs[sheet] = pd.read_excel(f"./dataguide/{file}", sheet_name=sheet)
        except Exception as e:
            print(f"오류 발생 - 파일: {file}, 시트: {sheet}, 원인: {e}")

for key, df in dfs.items():
    print(f"키: {key}, 데이터프레임 크기: {df.shape}")

키: depre_amort, 데이터프레임 크기: (113, 3775)
키: depre, 데이터프레임 크기: (113, 3775)
키: age, 데이터프레임 크기: (15, 3775)
키: n_acq_intangi, 데이터프레임 크기: (113, 3775)
키: tangi, 데이터프레임 크기: (113, 3775)
키: intangi, 데이터프레임 크기: (113, 3775)
키: cash, 데이터프레임 크기: (113, 3775)
키: payab, 데이터프레임 크기: (113, 3775)
키: invent, 데이터프레임 크기: (113, 3775)
키: const_receiv2, 데이터프레임 크기: (113, 3775)
키: receiv, 데이터프레임 크기: (113, 3775)
키: st_debt, 데이터프레임 크기: (113, 3775)
키: lt_debt, 데이터프레임 크기: (113, 3775)
키: t_debt, 데이터프레임 크기: (113, 3775)
키: const_receiv1, 데이터프레임 크기: (113, 3775)
키: amort, 데이터프레임 크기: (113, 3781)
키: defer_tax_debt, 데이터프레임 크기: (113, 3781)
키: common_mktcap, 데이터프레임 크기: (113, 3781)
키: prefer_mktcap, 데이터프레임 크기: (113, 3781)
키: asset, 데이터프레임 크기: (113, 3781)
키: exchange, 데이터프레임 크기: (113, 3775)
키: ind_code, 데이터프레임 크기: (113, 3775)
키: sales, 데이터프레임 크기: (113, 3781)
키: op_earn, 데이터프레임 크기: (113, 3781)
키: n_acq_tangi, 데이터프레임 크기: (113, 3781)


In [4]:
rows_to_drop = list(range(0, 13)) # 0 ~ 12

for k in dfs.keys():
  dfs[k].columns = dfs[k].iloc[7] # firmcode line을 column명으로 지정
  dfs[k] = dfs[k].drop(rows_to_drop, axis=0)
  dfs[k] = dfs[k].set_index("Symbol").reset_index().melt(
        id_vars=["Symbol"],
        var_name="firmcode",
        value_name=k,
    ).rename(columns={"Symbol": "date"})
  print(f"키: {k}, 데이터프레임 크기: {dfs[k].shape}")

C:\Users\DYK\AppData\Roaming\Python\Python311\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


키: depre_amort, 데이터프레임 크기: (377400, 3)


C:\Users\DYK\AppData\Roaming\Python\Python311\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


키: depre, 데이터프레임 크기: (377400, 3)


C:\Users\DYK\AppData\Roaming\Python\Python311\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


...

키: sales, 데이터프레임 크기: (378000, 3)


C:\Users\DYK\AppData\Roaming\Python\Python311\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


키: op_earn, 데이터프레임 크기: (378000, 3)


C:\Users\DYK\AppData\Roaming\Python\Python311\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


키: n_acq_tangi, 데이터프레임 크기: (378000, 3)


In [5]:
# import pickle

# with open("data.pkl", "wb") as f:
#   pickle.dump(dfs, f)

In [6]:
# import pickle
# with open("data.pkl", "rb") as f:
#   dfs = pickle.load(f)

In [113]:
df = dfs["asset"][["date", "firmcode"]]
for c in dfs.keys():
    df = pd.merge(df, dfs[c], on=["date", "firmcode"], how="left")


for v in df.columns:
    if v in ["date", "firmcode", "age", "exchange", "ind_code"]:
        continue
    df[v] = df[v].astype(float)

df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month

df = df[df["date"]!="2024-12-10"]
df = df[df["month"].isin([3, 6, 9, 12])].sort_values(["firmcode", "date"])




print(
    df.shape, 
    df["firmcode"].value_counts(dropna=False).value_counts(), # 3780개 회사는 모두 99개의 데이터를 가지고 있음
    df["date"].value_counts(dropna=False).value_counts(),  # 전체 99개일자마다 3780개 회사가 존재함
    df[['date', 'firmcode']].value_counts(dropna=False).value_counts(), # date, firmcode 조합은 unique함
)

(374220, 29) count
99    3780
Name: count, dtype: int64 count
3780    99
Name: count, dtype: int64 count
1    374220
Name: count, dtype: int64


In [114]:
for v in ["sales", "asset"]:
    print(v, df.groupby("firmcode")[v].sum().value_counts(dropna=False)[0])

sales 509
asset 208


In [75]:
df.groupby("firmcode").apply(lambda x: bfill_lag(ffill_lag(x["asset"]))).reset_index(drop=True).isna().sum(), df["asset"].isna().sum()

C:\Users\DYK\AppData\Local\Temp\ipykernel_30168\3244629390.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("firmcode").apply(lambda x: bfill_lag(ffill_lag(x["asset"]))).reset_index(drop=True).isna().sum(), df["asset"].isna().sum()


(152195, 180830)

In [76]:
df.groupby("firmcode").apply(lambda x: bfill_lag(ffill_lag(x["sales"]))).reset_index(drop=True).isna().sum(), df["sales"].isna().sum()

C:\Users\DYK\AppData\Local\Temp\ipykernel_30168\643066534.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("firmcode").apply(lambda x: bfill_lag(ffill_lag(x["sales"]))).reset_index(drop=True).isna().sum(), df["sales"].isna().sum()


(149586, 173084)

In [115]:
df["age"] = df.groupby("firmcode")["age"].ffill().bfill()
df["age"] = pd.to_datetime(df["age"], format="%Y%m%d").dt.strftime("%Y-%m-%d")

df = df.sort_values(["firmcode", "date"])
for v in ["asset", "sales", "op_earn", "depre_amort"]:
    df[v] = df.groupby("firmcode").apply(lambda x: bfill_lag(ffill_lag(x[v]))).reset_index(drop=True)\

# 건설업의 매출채권 성격
if "const_receiv1" in df.columns:
    df.loc[df["ind_code"] == "F", "receiv"] = df.loc[df["ind_code"] == "F", ["receiv", "const_receiv1", "const_receiv2"]].sum(axis=1)
    df = df.drop(columns=["const_receiv1", "const_receiv2"])

# Operating Earnings
if "ebitda" not in df.columns:
    df["ebitda"] = df["op_earn"] + df["depre_amort"]
    print(df[["ebitda", "op_earn", "depre_amort"]].isna().sum())
    df = df.drop(columns=["op_earn", "depre_amort"])

df["n_ppe"] = df["tangi"] + df["intangi"]
for v in ["sales", "asset", "ebitda", "n_ppe"]:
    df.loc[df[v] == 0, v] = None

# 날짜 오름차순 정렬이라서 shift(1)이 전날 데이터를 가져옴
df["asset_lag"] = df.groupby("firmcode")["asset"].shift(1) # = atq
df["cashflow"] = df["ebitda"] / df["asset_lag"]

C:\Users\DYK\AppData\Local\Temp\ipykernel_30168\3490777006.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["age"] = df.groupby("firmcode")["age"].ffill().bfill()
C:\Users\DYK\AppData\Local\Temp\ipykernel_30168\3490777006.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[v] = df.groupby("firmcode").apply(lambda x: bfill_lag(ffill_lag(x[v]))).reset_index(drop=True)\
C:\Users\DYK\AppData\Local\Temp\ipykernel_30168\3490777006.py:6: DeprecationWarning: DataFrameGroupBy.appl

ebitda         151978
op_earn        151846
depre_amort    151978
dtype: int64


C:\Users\DYK\AppData\Local\Temp\ipykernel_30168\3490777006.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[v] = df.groupby("firmcode").apply(lambda x: bfill_lag(ffill_lag(x[v]))).reset_index(drop=True)\


In [116]:
df[df["ebitda"].notna() & df["asset_lag"].notna()]["cashflow"].isna().sum()

0

In [117]:
df[["date", "asset", "asset_lag"]]

,date,asset,asset_lag
259000,2000-03-31,NaN,NaN
259001,2000-06-30,NaN,NaN
259002,2000-09-30,NaN,NaN
259003,2000-12-31,NaN,NaN
259004,2001-03-31,NaN,NaN
...,...,...,...
124494,2023-09-30,70533540980.000000,83072589000.000000
124495,2023-12-31,79423086000.000000,70533540980.000000
124496,2024-03-31,77359604000.000000,79423086000.000000
124497,2024-06-30,77565777000.000000,77359604000.000000


In [118]:
firm_obs = df.shape[0] / df["firmcode"].nunique()
print(df.shape[0], df["firmcode"].nunique(), firm_obs)

df = df[~df["ind_code"].isin(["K", "E", "D"])]
# K: 금융업
# D: 전기, 가스, 증기 및 공기 조절 공급업
# E: 수도, 하수, 및 폐기물 처리, 원료 재생업

firm_obs = df.shape[0] / df["firmcode"].nunique()
print(df.shape[0], df["firmcode"].nunique(), firm_obs)


374220 3780 99.0
332244 3356 99.0


In [121]:
df["firmcode"].value_counts(dropna=False).value_counts()

count
99    3356
Name: count, dtype: int64

In [119]:
df["payab_to_sales"] = df["payab"] / df["sales"]
df["payab_to_asset"] = df["payab"] / df["asset"]
df["receiv_to_sales"] = df["receiv"] / df["sales"]
df["receiv_to_asset"] = df["receiv"] / df["asset"]
df["invent_to_sales"] = df["invent"] / df["sales"]
df["invent_to_asset"] = df["invent"] / df["asset"]

In [122]:
lag_cols = [f"payab_to_sales_lag{i}" for i in range(1, 5)]
groups = df.groupby("firmcode")
for i in range(1, 5):
    df[lag_cols[i-1]] = groups["payab_to_sales"].shift(i)

df["sup_fin"] = df[lag_cols].mean(axis=1)
df = df.drop(columns=lag_cols)
df["sup_fin_tercile"] = pd.qcut(df["sup_fin"], 3, labels=["low", "mid", "top"])

In [123]:
lag_cols = [f"cashflow_lag{i}" for i in range(1, 5)]
groups = df.groupby("firmcode")
for i in range(1, 5):
    df[lag_cols[i-1]] = groups["cashflow"].shift(i)

df["delta_cashflow"] = df["cashflow"] - df[lag_cols].mean(axis=1) 
df = df.drop(columns=lag_cols)

In [124]:
df["log_sales"] = df["sales"].astype(float).apply(np.log) # default of base is e
df["size"] = (1+df["asset_lag"].astype(float)).apply(np.log)
df["book_lev"] = (df["st_debt"] + df["lt_debt"]) / df["asset"]

df["st_debt_to_asset"] = df["st_debt"] / df["asset"]
df["st_debt_to_sales"] = df["lt_debt"] / df["sales"]
df["st_asset"] = df["receiv"] + df["invent"]

df["capex"] = df["n_acq_tangi"] + df["n_acq_intangi"]
df["investment"] = df["capex"] / df.groupby("firmcode")["n_ppe"].shift(1)
df["n_ppe_to_asset"] = df["n_ppe"] / df["asset"]

df["q"] = (df["common_mktcap"] + df["prefer_mktcap"] + df["st_debt"] + df["lt_debt"]+ df["defer_tax_debt"]) / df["asset_lag"]

In [125]:
df["drop_size_dummy"] = df["asset_lag"] > df["asset_lag"].quantile(0.66)

In [126]:
t = df.groupby("firmcode")["asset"].count()
df = df[~df["firmcode"].isin(t[t < DEL_UNDER_YEAR].index)]

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 317790 entries, 82600 to 124498
Data columns (total 49 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   date              317790 non-null  datetime64[ns]
 1   firmcode          317790 non-null  object        
 2   depre             180257 non-null  float64       
 3   age               317790 non-null  object        
 4   n_acq_intangi     205129 non-null  float64       
 5   tangi             173283 non-null  float64       
 6   intangi           173283 non-null  float64       
 7   cash              173283 non-null  float64       
 8   payab             173249 non-null  float64       
 9   invent            173249 non-null  float64       
 10  receiv            171789 non-null  float64       
 11  st_debt           173249 non-null  float64       
 12  lt_debt           173249 non-null  float64       
 13  t_debt            173283 non-null  float64       
 14  amort

In [128]:
print(
    df["firmcode"].nunique(),
    df.shape,
    df["cashflow"].isna().sum(),
    df["cashflow"].isna().sum() / df.shape[0],
    sep="\n"
)

3210
(317790, 49)
149603
0.4707605651530885


In [129]:
# firm 마다 분기별 데이터가 있는지 확인
# 모든 분기에 대해 데이터가 있는 firmcode만 남기기
(df["firmcode"].value_counts() == 99).sum()

3210

In [130]:
df.columns

Index(['date', 'firmcode', 'depre', 'age', 'n_acq_intangi', 'tangi', 'intangi',
       'cash', 'payab', 'invent', 'receiv', 'st_debt', 'lt_debt', 't_debt',
       'amort', 'defer_tax_debt', 'common_mktcap', 'prefer_mktcap', 'asset',
       'exchange', 'ind_code', 'sales', 'n_acq_tangi', 'year', 'month',
       'ebitda', 'n_ppe', 'asset_lag', 'cashflow', 'payab_to_sales',
       'payab_to_asset', 'receiv_to_sales', 'receiv_to_asset',
       'invent_to_sales', 'invent_to_asset', 'sup_fin', 'sup_fin_tercile',
       'delta_cashflow', 'log_sales', 'size', 'book_lev', 'st_debt_to_asset',
       'st_debt_to_sales', 'st_asset', 'capex', 'investment', 'n_ppe_to_asset',
       'q', 'drop_size_dummy'],
      dtype='object')

In [131]:
df["cashflow"] = df["cashflow"].astype(float)
groups = df.reset_index(drop=True).groupby(["firmcode", "month"])


lag_cols = [f"cashflow_lag{i}" for i in range(1, 4*int(MAX_YEAR+1))]
for i in range(1, 4*int(MAX_YEAR+1)):
    df[lag_cols[i-1]] = groups["cashflow"].shift(i)


skip_quater = 4 # previous 1 year = 4 quaters
quater_lag = [None] + [lag_cols[i::4][:int(MAX_YEAR)] for i in range(skip_quater-1, skip_quater-1+4)]
for i, v in enumerate(quater_lag):
    if not i: continue
    print(f"Q{i}", quater_lag[i])
    df[f"cashflow_q{i}_mean"] = df[quater_lag[i]].mean(axis=1)
    df[f"cashflow_q{i}_cnt"] = df[quater_lag[i]].notna().sum(axis=1)

df["main_quater"] = df[[f"cashflow_q{i}_mean" for i in range(1, 5)]].idxmax(axis=1).str[-6].astype(float)
df["main_quater_cnt"] = df[[f"cashflow_q{i}_cnt" for i in range(1, 5)]].sum(axis=1) >= ((MAX_YEAR) * 4 - 1)

df = df.drop(columns=lag_cols)

Q1 ['cashflow_lag4', 'cashflow_lag8', 'cashflow_lag12']
Q2 ['cashflow_lag5', 'cashflow_lag9', 'cashflow_lag13']
Q3 ['cashflow_lag6', 'cashflow_lag10', 'cashflow_lag14']
Q4 ['cashflow_lag7', 'cashflow_lag11', 'cashflow_lag15']


C:\Users\DYK\AppData\Local\Temp\ipykernel_30168\802554432.py:18: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df["main_quater"] = df[[f"cashflow_q{i}_mean" for i in range(1, 5)]].idxmax(axis=1).str[-6].astype(float)


In [132]:
df["main_quater_cnt"].value_counts()

main_quater_cnt
False    280591
True      37199
Name: count, dtype: int64

In [144]:
print("firm observations", df[(df["main_quater_cnt"]==True)]["firmcode"].nunique())
df[(df["main_quater_cnt"]==True)]["ind_code"].value_counts()

firm observations 1494


ind_code
C    23037
J     3844
G     2921
M     2616
F     1397
H      409
N      177
I      160
L      122
A      120
P       96
R       90
S        1
Name: count, dtype: int64

In [145]:
summary = df.loc[(df["main_quater_cnt"]==True), [
     "log_sales", "size", "q", "cashflow", "cash", "book_lev", 
    "st_debt_to_asset", "st_debt_to_sales", 
    "payab_to_sales", "payab_to_asset", "receiv_to_sales", "receiv_to_asset", "invent_to_sales", "invent_to_asset", 
    "investment", "n_ppe_to_asset"
]].describe(
    include="all"
).T

summary["count"] = summary["count"].astype(int)
summary[["count", "mean", "50%", "std", "min", "25%", "75%", "max"]]

,count,mean,50%,std,min,25%,75%,max
log_sales,25412,23.903550,23.858097,1.840397,10.596635,22.814518,24.889445,30.876334
size,27450,25.918971,25.707908,1.661303,16.692317,24.887475,26.623343,33.910172
q,13447,4.154108,0.757492,34.763171,0.000039,0.227914,2.086722,3368.986976
cashflow,25406,0.011636,0.010886,0.098968,-14.484434,0.000569,0.024928,1.030641
cash,20476,41918119471.613106,5347307500.000000,184412281064.332489,-9137649000.000000,1642886000.000000,16607666500.000000,7049394000000.000000
book_lev,15165,1.160372,0.100429,9.731473,0.000000,0.010731,0.529486,343.517369
st_debt_to_asset,15165,0.642614,0.058040,4.449194,0.000000,0.004522,0.324940,199.856054
st_debt_to_sales,14085,-12124.687788,0.068179,1445354.398664,-171534800.000000,0.000409,0.958775,175928.956966
payab_to_sales,14085,-22297.782286,0.178642,2653667.892102,-314937800.000000,0.018602,0.996092,176179.925000
payab_to_asset,15165,0.482658,0.026117,7.772372,0.000000,0.003147,0.133470,333.018647


In [134]:
df[(df["main_quater_cnt"]==True) & (df["cashflow"].notna())]["drop_size_dummy"].value_counts()

drop_size_dummy
False    13592
True     11814
Name: count, dtype: int64

In [135]:
13592/(13592+11814)

0.5349917342360072

In [136]:
print("firm observations", df[(df["main_quater_cnt"]==True) & (df["cashflow"].notna())]["firmcode"].nunique())
df[(df["main_quater_cnt"]==True) & (df["cashflow"].notna())]["ind_code"].value_counts()

firm observations 1151


ind_code
C    15605
J     2550
G     2111
M     1743
F     1096
H      291
N      154
I      112
L       81
A       70
R       60
P       47
Name: count, dtype: int64

In [138]:
summary = df.loc[(df["main_quater_cnt"]==True) & (df["cashflow"].notna()), [
     "log_sales", "size", "q", "cashflow", "cash", "book_lev", 
    "st_debt_to_asset", "st_debt_to_sales", 
    "payab_to_sales", "payab_to_asset", "receiv_to_sales", "receiv_to_asset", "invent_to_sales", "invent_to_asset", 
    "investment", "n_ppe_to_asset"
]].describe(
    include="all"
).T

summary["count"] = summary["count"].astype(int)
summary[["count", "mean", "50%", "std", "min", "25%", "75%", "max"]]

,count,mean,50%,std,min,25%,75%,max
log_sales,25020,23.907220,23.864866,1.837271,10.596635,22.818234,24.893361,30.876334
size,25406,26.037143,25.794795,1.625010,16.692317,24.992774,26.697158,33.910172
q,12541,3.606289,0.721974,19.425051,0.000039,0.212788,1.899640,525.574284
cashflow,25406,0.011636,0.010886,0.098968,-14.484434,0.000569,0.024928,1.030641
cash,14082,43352057598.366707,5327333500.000000,192773221105.731140,-9137649000.000000,1652749750.000000,16888656500.000000,7049394000000.000000
book_lev,14036,1.084191,0.091297,9.962716,0.000000,0.009520,0.469391,343.517369
st_debt_to_asset,14036,0.587133,0.052207,4.435291,0.000000,0.004046,0.288548,199.856054
st_debt_to_sales,13966,-12228.065606,0.066966,1451499.051199,-171534800.000000,0.000396,0.955962,175928.956966
payab_to_sales,13966,-22487.865649,0.176986,2664949.459952,-314937800.000000,0.018376,0.991379,176179.925000
payab_to_asset,14036,0.473429,0.023405,8.067403,0.000000,0.002767,0.119951,333.018647


In [141]:
print("firm observations", df[(df["main_quater_cnt"]==True) & (df["cashflow"].notna()) & (df["sup_fin_tercile"]=="top")]["firmcode"].nunique())
df[(df["main_quater_cnt"]==True) & (df["cashflow"].notna()) & (df["sup_fin_tercile"]=="top")]["ind_code"].value_counts()

firm observations 349


ind_code
C    3173
F     448
G     424
J     232
M     225
H      79
N      19
P      16
A      10
I       7
L       4
Name: count, dtype: int64

In [140]:
summary = df.loc[(df["main_quater_cnt"]==True) & (df["cashflow"].notna()) & (df["sup_fin_tercile"]=="top"), [
     "log_sales", "size", "q", "cashflow", "cash", "book_lev", 
    "st_debt_to_asset", "st_debt_to_sales", 
    "payab_to_sales", "payab_to_asset", "receiv_to_sales", "receiv_to_asset", "invent_to_sales", "invent_to_asset", 
    "investment", "n_ppe_to_asset"
]].describe(
    include="all"
).T

summary["count"] = summary["count"].astype(int)
summary[["count", "mean", "50%", "std", "min", "25%", "75%", "max"]]

,count,mean,50%,std,min,25%,75%,max
log_sales,4807,22.947375,23.021254,1.833790,10.596635,21.930290,24.140758,28.394941
size,4880,25.498780,25.323651,1.263957,20.822787,24.659456,26.261181,31.575067
q,3940,8.789558,1.921461,33.301493,0.003079,0.878354,4.984139,525.574284
cashflow,4880,0.005098,0.006176,0.044399,-0.586435,-0.004802,0.018236,0.933472
cash,4279,103210838021.666275,10706286000.000000,327506428073.231201,-2099780000.000000,2775848000.000000,49308066000.000000,7049394000000.000000
book_lev,4269,2.878481,0.513850,17.741251,0.000000,0.100052,1.660806,343.517369
st_debt_to_asset,4269,1.524434,0.320241,7.868739,0.000000,0.064799,1.047478,199.856054
st_debt_to_sales,4243,-40252.574482,1.046735,2633398.384287,-171534800.000000,0.034835,6.300953,175928.956966
payab_to_sales,4243,-74020.722343,2.157769,4834913.135067,-314937800.000000,1.036434,5.942739,176179.925000
payab_to_asset,4269,1.469632,0.233660,14.571247,0.000000,0.096941,0.530473,333.018647


In [142]:
df.to_csv("summary_stat_delta_cashflow.csv", index=False, encoding="utf8")

In [143]:
df.columns

Index(['date', 'firmcode', 'depre', 'age', 'n_acq_intangi', 'tangi', 'intangi',
       'cash', 'payab', 'invent', 'receiv', 'st_debt', 'lt_debt', 't_debt',
       'amort', 'defer_tax_debt', 'common_mktcap', 'prefer_mktcap', 'asset',
       'exchange', 'ind_code', 'sales', 'n_acq_tangi', 'year', 'month',
       'ebitda', 'n_ppe', 'asset_lag', 'cashflow', 'payab_to_sales',
       'payab_to_asset', 'receiv_to_sales', 'receiv_to_asset',
       'invent_to_sales', 'invent_to_asset', 'sup_fin', 'sup_fin_tercile',
       'delta_cashflow', 'log_sales', 'size', 'book_lev', 'st_debt_to_asset',
       'st_debt_to_sales', 'st_asset', 'capex', 'investment', 'n_ppe_to_asset',
       'q', 'drop_size_dummy', 'cashflow_q1_mean', 'cashflow_q1_cnt',
       'cashflow_q2_mean', 'cashflow_q2_cnt', 'cashflow_q3_mean',
       'cashflow_q3_cnt', 'cashflow_q4_mean', 'cashflow_q4_cnt', 'main_quater',
       'main_quater_cnt'],
      dtype='object')